<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# CAPSTONE : Ubisoft's Assassin's Creed IV – Predicting Topics From Reviews

# Part 1 – Data Extraction

This notebook is on the data extraction only.

In [1]:
# Importing all libraries used: 

import requests
import pandas as pd      
import re
import os
from time import sleep
import random

### Assassin's Creed

In [2]:
# Using API to retrieve app reviews of Assassins Creed IV: Black Flag. ID of game is 242050
ass_creed_res = requests.get(url='https://store.steampowered.com/appreviews/242050?json=1').json()

In [3]:
# Identifying the keys of the dictionary. 
ass_creed_res.keys()

dict_keys(['success', 'query_summary', 'reviews', 'cursor'])

In [4]:
# Accessing Query Summary: Note that the current reviews are only the first 20.
ass_creed_res['query_summary']

{'num_reviews': 20,
 'review_score': 8,
 'review_score_desc': 'Very Positive',
 'total_positive': 19517,
 'total_negative': 2765,
 'total_reviews': 22282}

In [5]:
# Cursor will be needed to access then next 20 reviews.
ass_creed_res['cursor']

'AoIIPwYYanzn0qIE'

In [6]:
# Trying to access the next 20 reviews.
requests.get(url='https://store.steampowered.com/appreviews/242050?json=1&cursor=AoIIPxCyEH66sZwE').json()

{'success': 1,
 'query_summary': {'num_reviews': 20},
 'reviews': [{'recommendationid': '143362135',
   'author': {'steamid': '76561198172439922',
    'num_games_owned': 0,
    'num_reviews': 18,
    'playtime_forever': 2353,
    'playtime_last_two_weeks': 0,
    'playtime_at_review': 780,
    'last_played': 1691468856},
   'language': 'english',
   'review': "bought a game through steam, so much to my surprise the clowns at ubisoft make you find the cd key and input into their crap system too, since we all know ubisoft is prob the worst programmer don't expect them to intergrate with steam, also love having to sign into ubisoft every time i want to play since those morons don't have a remember me or keep me signed in feature.",
   'timestamp_created': 1691114746,
   'timestamp_updated': 1691114746,
   'voted_up': False,
   'votes_up': 3,
   'votes_funny': 0,
   'weighted_vote_score': '0.560439586639404297',
   'comment_count': 0,
   'steam_purchase': True,
   'received_for_free': Fals

In [7]:
# Defining a function to automate this process

def get_reviews(appid, params={'json':1}):
    url = 'https://store.steampowered.com/appreviews/'
    response = requests.get(url=url+appid, params=params)
    return response.json()

def get_n_reviews(appid, n=100):
    reviews = []
    cursor = '*'
    params = {
            'json' : 1,
            'filter' : 'all',
            'language' : 'english',
            'day_range' : 9223372036854775807,
            'review_type' : 'all',
            'purchase_type' : 'all'
            }

    while n > 0:
        params['cursor'] = cursor.encode()
        params['num_per_page'] = min(100, n)
        n -= 100

        response = get_reviews(appid, params)
        cursor = response['cursor']
        reviews += response['reviews']

        sleep(random.uniform(0.5, 2))

        if len(response['reviews']) < 100: break

    return reviews

In [8]:
# Converting results of all assassins creed reviews into a dataframe
# Note: At point of extraction, there are 22264 reviews for the game
ass_creed = pd.DataFrame(get_n_reviews('242050', n =22264))

In [9]:
ass_creed.head()

,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location,timestamp_dev_responded,developer_response
0,85460916,"{'steamid': '76561198156016347', 'num_games_ow...",english,The only game where i avoid fast travel.,1611779242,1611779242,True,3748,185,0.986814916133880615,11,True,False,False,False,,NaN,NaN
1,76223418,"{'steamid': '76561198149047566', 'num_games_ow...",english,This is the Best Assassins Creed game and Prob...,1600521179,1600521179,True,2558,37,0.979318380355834961,11,True,False,False,False,,NaN,NaN
2,69682784,"{'steamid': '76561197990642312', 'num_games_ow...",english,"Best game ever, iv'e been playing from day one...",1590260976,1590260976,True,2190,227,0.979192793369293213,0,True,False,False,False,,NaN,NaN
3,88206852,"{'steamid': '76561198982554607', 'num_games_ow...",english,Shanties Before Panties,1615404962,1615404962,True,2174,897,0.974362730979919434,11,True,False,False,False,,NaN,NaN
4,71445029,"{'steamid': '76561198849119685', 'num_games_ow...",english,Best part of the game are the sea shanties. Lo...,1593217346,1593217346,True,1041,240,0.958593249320983887,0,True,False,False,False,,NaN,NaN


In [10]:
# Turning Author into a dictionary
ac_auth = dict(enumerate(ass_creed['author'], 1))

ac_auth

{1: {'steamid': '76561198156016347',
  'num_games_owned': 267,
  'num_reviews': 21,
  'playtime_forever': 2856,
  'playtime_last_two_weeks': 0,
  'playtime_at_review': 2856,
  'last_played': 1518982944},
 2: {'steamid': '76561198149047566',
  'num_games_owned': 0,
  'num_reviews': 52,
  'playtime_forever': 1838,
  'playtime_last_two_weeks': 0,
  'playtime_at_review': 1838,
  'last_played': 1593601946},
 3: {'steamid': '76561197990642312',
  'num_games_owned': 0,
  'num_reviews': 2,
  'playtime_forever': 117391,
  'playtime_last_two_weeks': 77,
  'playtime_at_review': 90196,
  'last_played': 1693020785},
 4: {'steamid': '76561198982554607',
  'num_games_owned': 77,
  'num_reviews': 4,
  'playtime_forever': 4719,
  'playtime_last_two_weeks': 0,
  'playtime_at_review': 4036,
  'last_played': 1617472093},
 5: {'steamid': '76561198849119685',
  'num_games_owned': 0,
  'num_reviews': 10,
  'playtime_forever': 1592,
  'playtime_last_two_weeks': 0,
  'playtime_at_review': 1177,
  'last_played'

In [11]:
#Turning author dictionary into a dataframe
ac_auth_df = pd.DataFrame.from_dict(ac_auth, orient = 'index').reset_index(drop=True)
ac_auth_df.head()

,steamid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played
0,76561198156016347,267,21,2856,0,2856.0,1518982944
1,76561198149047566,0,52,1838,0,1838.0,1593601946
2,76561197990642312,0,2,117391,77,90196.0,1693020785
3,76561198982554607,77,4,4719,0,4036.0,1617472093
4,76561198849119685,0,10,1592,0,1177.0,1644543449


In [12]:
# Joining the two dataframes together
ass_creed = ac_auth_df.join(ass_creed)
ass_creed.head()

,steamid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played,recommendationid,author,language,...,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location,timestamp_dev_responded,developer_response
0,76561198156016347,267,21,2856,0,2856.0,1518982944,85460916,"{'steamid': '76561198156016347', 'num_games_ow...",english,...,185,0.986814916133880615,11,True,False,False,False,,NaN,NaN
1,76561198149047566,0,52,1838,0,1838.0,1593601946,76223418,"{'steamid': '76561198149047566', 'num_games_ow...",english,...,37,0.979318380355834961,11,True,False,False,False,,NaN,NaN
2,76561197990642312,0,2,117391,77,90196.0,1693020785,69682784,"{'steamid': '76561197990642312', 'num_games_ow...",english,...,227,0.979192793369293213,0,True,False,False,False,,NaN,NaN
3,76561198982554607,77,4,4719,0,4036.0,1617472093,88206852,"{'steamid': '76561198982554607', 'num_games_ow...",english,...,897,0.974362730979919434,11,True,False,False,False,,NaN,NaN
4,76561198849119685,0,10,1592,0,1177.0,1644543449,71445029,"{'steamid': '76561198849119685', 'num_games_ow...",english,...,240,0.958593249320983887,0,True,False,False,False,,NaN,NaN


In [13]:
# Dropping the following columns due to repetition or unncessary.
ass_creed.drop(columns = ['author', 'timestamp_dev_responded', 'developer_response', 'steam_china_location'], inplace = True)
ass_creed.head()

,steamid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played,recommendationid,language,review,...,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china
0,76561198156016347,267,21,2856,0,2856.0,1518982944,85460916,english,The only game where i avoid fast travel.,...,1611779242,True,3748,185,0.986814916133880615,11,True,False,False,False
1,76561198149047566,0,52,1838,0,1838.0,1593601946,76223418,english,This is the Best Assassins Creed game and Prob...,...,1600521179,True,2558,37,0.979318380355834961,11,True,False,False,False
2,76561197990642312,0,2,117391,77,90196.0,1693020785,69682784,english,"Best game ever, iv'e been playing from day one...",...,1590260976,True,2190,227,0.979192793369293213,0,True,False,False,False
3,76561198982554607,77,4,4719,0,4036.0,1617472093,88206852,english,Shanties Before Panties,...,1615404962,True,2174,897,0.974362730979919434,11,True,False,False,False
4,76561198849119685,0,10,1592,0,1177.0,1644543449,71445029,english,Best part of the game are the sea shanties. Lo...,...,1593217346,True,1041,240,0.958593249320983887,0,True,False,False,False


In [14]:
# Checking for duplicated rows
ass_creed.duplicated().sum()

0

In [16]:
# Creating directory and saving the Assassin's Creed Reviews.
if not os.path.exists('../data'):
    os.makedirs('../data') 
ass_creed.to_csv('../data/assassins_creed_reviews.csv', index= False)

Moving on to the next notebook for the main start of the project so we don't accidentally run our data extraction process again.